In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U gdown

In [3]:
!gdown --no-cookies 1a_wm8AyQv_EnrslEIQqI50R4_NtIcZqH
!gdown --no-cookies 1JDxY55PMhROW8D0WHk27HPtKBeQgF98M
!gdown --no-cookies 1CVelB69EvomYKi76gEd90Ib5bywKDWvE
!gdown --no-cookies 1OcBTIDI5qxv7vIhgA1gXC6Cy-FsQY-US
!gdown --no-cookies 1abTpCI-nHaTkvUKaUsOLySIvrmaJHvOb

Downloading...
From (uriginal): https://drive.google.com/uc?id=1a_wm8AyQv_EnrslEIQqI50R4_NtIcZqH
From (redirected): https://drive.google.com/uc?id=1a_wm8AyQv_EnrslEIQqI50R4_NtIcZqH&confirm=t&uuid=14efefc7-e442-43e3-88f7-1fd26820e6f0
To: /kaggle/working/F001E.csv
100%|███████████████████████████████████████| 1.14G/1.14G [00:09<00:00, 126MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1JDxY55PMhROW8D0WHk27HPtKBeQgF98M
From (redirected): https://drive.google.com/uc?id=1JDxY55PMhROW8D0WHk27HPtKBeQgF98M&confirm=t&uuid=cd18f434-e719-443e-ba75-be35f6ffaee4
To: /kaggle/working/F001D.csv
100%|███████████████████████████████████████| 1.17G/1.17G [00:09<00:00, 118MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1CVelB69EvomYKi76gEd90Ib5bywKDWvE
From (redirected): https://drive.google.com/uc?id=1CVelB69EvomYKi76gEd90Ib5bywKDWvE&confirm=t&uuid=da2b7dce-cbaa-431f-ade1-656e125ca867
To: /kaggle/working/F001C.csv
100%|███████████████████████████████████████| 1.1

In [4]:
!gdown --no-cookies 13v_GItwKUeB7HN-dQxEL9rMqXgiEFNT2

Downloading...
From: https://drive.google.com/uc?id=13v_GItwKUeB7HN-dQxEL9rMqXgiEFNT2
To: /kaggle/working/ERB Oyster Conditioning - Results Winter 2019.xlsx
100%|██████████████████████████████████████| 38.5k/38.5k [00:00<00:00, 45.4MB/s]


In [5]:
import pandas as pd
import numpy as np
import glob

In [6]:
files = glob.glob('*.csv')
codes = ['A','B','C','D','E']
files_dict = {code: next(file for file in files if code in file) for code in codes}
files_dict

{'A': 'F001A.csv',
 'B': 'F001B.csv',
 'C': 'F001C.csv',
 'D': 'F001D.csv',
 'E': 'F001E.csv'}

In [7]:
data_frames = {}

for code, filename in files_dict.items():
    data_frames[code] = pd.read_csv(filename, header=16, index_col=0)

In [8]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props

In [9]:
for code, df in data_frames.items():
    df = reduce_mem_usage(df)

Memory usage of properties dataframe is : 1181.276969909668  MB
******************************
Column:  CH_1
dtype before:  int64
dtype after:  int32
******************************
******************************
Column:  CH_2
dtype before:  int64
dtype after:  int32
******************************
******************************
Column:  CH_3
dtype before:  int64
dtype after:  int32
******************************
******************************
Column:  CH_4
dtype before:  int64
dtype after:  int32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  708.7661819458008  MB
This is  60.0 % of the initial size
Memory usage of properties dataframe is : 1181.250343322754  MB
******************************
Column:  CH_1
dtype before:  int64
dtype after:  int32
******************************
******************************
Column:  CH_2
dtype before:  int64
dtype after:  int32
******************************
******************************
Column:  CH_3
dtype before

In [10]:
from sklearn.preprocessing import StandardScaler
import gc
combined_df = pd.DataFrame()


In [11]:
genders = pd.read_excel('/kaggle/working/ERB Oyster Conditioning - Results Winter 2019.xlsx')
genders

,Date,ElapD,ID,PTClosed,Micro,MicroC,Sex
0,2019-03-01,7,1,5.474769,275,275,male (n=9)
1,2019-03-02,8,1,13.150579,164,439,male (n=9)
2,2019-03-03,9,1,10.518403,233,672,male (n=9)
3,2019-03-04,10,1,6.981944,208,880,male (n=9)
4,2019-03-05,11,1,10.140046,148,1028,male (n=9)
...,...,...,...,...,...,...,...
443,2019-03-24,30,20,11.766319,102,2689,male (n=9)
444,2019-03-25,31,20,16.346991,11,2700,male (n=9)
445,2019-03-26,32,20,46.827894,21,2721,male (n=9)
446,2019-03-27,33,20,28.298148,203,2924,male (n=9)


In [12]:
import re

genders_iterator = []

for id, group in genders.groupby('ID'):
    if any(re.search(r'\bfemale\b', value) for value in group['Sex'].values):
        genders_iterator.append(0)
    else:
        genders_iterator.append(1)


In [13]:
genders_iterator, len(genders_iterator) # 5, 11, 13, 19 -> B_1, C_3 , D_1 , E_3

([1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1], 16)

In [14]:
genders_iterator = iter(genders_iterator)

In [15]:
exluded = ['B_1', 'C_3' , 'D_1' , 'E_3']

In [16]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import gc

code_dfs = []

for code, df in list(data_frames.items()): 
    for i in range(1, len(df.columns) + 1):
        idx = f'{code}_{i}'
        print(idx)
        if idx in exluded : 
            continue
        gc.collect()
        scaler = StandardScaler()
        code_df = pd.DataFrame()
        code_df['ID'] = [f'{code}_{i}'] * len(df[f'CH_{i}'])
        code_df['Sequence'] = scaler.fit_transform(df[f'CH_{i}'].values.reshape(-1, 1))
        code_df['label'] = [next(genders_iterator)] * len(df[f'CH_{i}'])
        code_df = reduce_mem_usage(code_df)
        code_dfs.append(code_df)
    del data_frames[code]

combined_df = pd.concat(code_dfs, ignore_index=True)

combined_df.reset_index(inplace=True, drop=True)

del code_dfs
gc.collect()

A_1
Memory usage of properties dataframe is : 708.7663040161133  MB
******************************
Column:  Sequence
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  label
dtype before:  int64
dtype after:  uint8
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  383.9151372909546  MB
This is  54.16667456050882 % of the initial size
A_2
Memory usage of properties dataframe is : 708.7663040161133  MB
******************************
Column:  Sequence
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  label
dtype before:  int64
dtype after:  uint8
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  383.9151372909546  MB
This is  54.16667456050882 % of the initial size
A_3
Memory usage of properties dataframe is : 708.7663040161133  MB
******************************
Column:  Sequence
dtype bef

0

In [17]:
combined_df = reduce_mem_usage(combined_df)

Memory usage of properties dataframe is : 6142.415271759033  MB
******************************
Column:  Sequence
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  label
dtype before:  uint8
dtype after:  uint8
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  6142.415271759033  MB
This is  100.0 % of the initial size


In [18]:
6142/8.29

740.8926417370326

In [19]:
combined_df

,ID,Sequence,label
0,A_1,-3.237192,1
1,A_1,-3.238868,1
2,A_1,-3.238533,1
3,A_1,-3.238198,1
4,A_1,-3.240210,1
...,...,...,...
495445311,E_4,-1.528964,1
495445312,E_4,-1.523105,1
495445313,E_4,-1.523756,1
495445314,E_4,-1.530700,1


In [20]:
combined_df.to_csv('train_df2_after_norm_mem_reduce.csv',index=False)

In [21]:
import os
os.remove('/kaggle/working/ERB Oyster Conditioning - Results Winter 2019.xlsx')
[os.remove(f) for f in files]

[None, None, None, None, None]

In [22]:
combined_df[combined_df['ID'] == 'A_2']

,ID,Sequence,label
30966467,A_2,-3.325542,0
30966468,A_2,-3.326074,0
30966469,A_2,-3.325010,0
30966470,A_2,-3.326074,0
30966471,A_2,-3.325808,0
...,...,...,...
61932929,A_2,-1.029373,0
61932930,A_2,-1.030171,0
61932931,A_2,-1.030437,0
61932932,A_2,-1.027245,0


In [23]:
combined_df.shape[0]  / 30966467

15.999413688361672

In [24]:
nrows = combined_df.shape[0]
nseconds = nrows/10
nhours = nseconds / 3600
ndays = nhours/24
ndays , nhours

(573.4320787037037, 13762.369888888888)

In [25]:
13762/8

1720.25